OBJECT
This database is to provide a tool for a particular challenge run in the videogame "Pokemon Emerald" where the ROM has been altered to disable experience gain. In this run, caught Pokemon will be stuck with the moves they are first caught with. This database has tables for all in-game locations and includes the available Pokemon at that location, their maximum possible level at encounter, and their four move slots in respect to their level. 

This data exists online, but would involve searching the move tables of every individual creature as it came up on the current in game location. It is also a practice project for creating databases from disparate data sources and for advanced web scraping.

GETTING THE LEARNSET DATA
1. scrape the names, in lowercase, of the Pokemon in Hoenn Pokedex.
   https://pokemondb.net/pokedex/game/ruby-sapphire-emerald
   list: names
2. loop the pages https://pokemondb.net/pokedex/lotad/moves/3
   with a variable iterating over names in place of lotad.
3. For each name in names...
   Scrape the 'Moves learnt by level up' table ('Lv.' and 'Move') from page.
   Create a table 'name' containing 'Name', 'Level' and 'Move'.
   
GETTING THE LOCATION DATA
1. scrape the locations in lowercase with no spaces.
   https://pokemondb.net/location#tab-hoenn
   list: locations
2. loop the pages https://www.serebii.net/pokearth/hoenn/3rd/route102.shtml    with a variable 'loc' in place of route102.
3. For each loc in locations...
   Scrape the 'Pokemon Emerald' container (Pokemon name, 'Max Level') in the    'Standard Walking' container
   create a table 'location' containing 'Name', 'MaxLv', 'Method' (which should be set equal to "grass"),
   'Move1', 'Move2', 'Move3', 'Move4'.
   
BRINGING THE LOCATION AND LEARNSET DATA TOGETHER

In [3]:
#imports including created csvs, db
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

conn = sqlite3.connect('Emerald_Encounters')
cursor = conn.cursor()

locations = pd.read_csv("locationlist.csv")['0'].tolist()
pokemon = pd.read_csv("pokemonlist.csv")['0'].tolist()

# Locations List

In [65]:
url_loc = 'https://pokemondb.net/location#tab-hoenn'
response_loc = requests.get(url_loc)
soup_loc = BeautifulSoup(response_loc.content, "html.parser")

In [66]:
locations = soup_loc.find(id = 'loc-hoenn').text.strip()
print(locations)

Route 101
Route 102
Route 103
Route 104
Route 105
Route 106
Route 107
Route 108
Route 109
Route 110
Route 111
Route 112
Route 113
Route 114
Route 115


Route 116
Route 117
Route 118
Route 119
Route 120
Route 121
Route 122
Route 123
Route 124
Route 125
Route 126
Route 127
Route 128
Route 129
Route 130


Route 131
Route 132
Route 133
Route 134
Abandoned Ship
Altering Cave
Artisan Cave
Battle Frontier
Battle Resort
Battle Tower
Birth Island
Cave of Origin
Desert Underpass
Dewford Town
Ever Grande City


Fallarbor Town
Faraway Island
Fiery Path
Fortree City
Granite Cave
Jagged Pass
Lavaridge Town
Lilycove City
Littleroot Town
Marine Cave
Mauville City
Meteor Falls
Mirage Island
Mirage Spots
Mirage Tower


Mossdeep City
Mt. Chimney
Mt. Pyre
New Mauville
Oldale Town
Pacifidlog Town
Petalburg City
Petalburg Woods
Roaming Hoenn
Rustboro City
Rusturf Tunnel
Safari Zone
Scorched Slab
Sea Mauville
Seafloor Cavern


Sealed Chamber
Shoal Cave
Sky Pillar
Slateport City
Sootopolis City
Southern Islan

In [67]:
locations = locations.lower().replace(' ', '')
locations = locations.split("\n")

In [68]:
locations = list(filter(None, locations))
print(locations)

['route101', 'route102', 'route103', 'route104', 'route105', 'route106', 'route107', 'route108', 'route109', 'route110', 'route111', 'route112', 'route113', 'route114', 'route115', 'route116', 'route117', 'route118', 'route119', 'route120', 'route121', 'route122', 'route123', 'route124', 'route125', 'route126', 'route127', 'route128', 'route129', 'route130', 'route131', 'route132', 'route133', 'route134', 'abandonedship', 'alteringcave', 'artisancave', 'battlefrontier', 'battleresort', 'battletower', 'birthisland', 'caveoforigin', 'desertunderpass', 'dewfordtown', 'evergrandecity', 'fallarbortown', 'farawayisland', 'fierypath', 'fortreecity', 'granitecave', 'jaggedpass', 'lavaridgetown', 'lilycovecity', 'littleroottown', 'marinecave', 'mauvillecity', 'meteorfalls', 'mirageisland', 'miragespots', 'miragetower', 'mossdeepcity', 'mt.chimney', 'mt.pyre', 'newmauville', 'oldaletown', 'pacifidlogtown', 'petalburgcity', 'petalburgwoods', 'roaminghoenn', 'rustborocity', 'rusturftunnel', 'safar

In [69]:
#delete locations with no encounters, postgame, or contain gift pokemon
notneeded = ['battlefrontier', 'mt.chimney', 'oldaletown',
             'fallarbortown', 'lavaridgetown', 'mauvillecity',
             'sstidal', 'trainerhill', 'verdanturftown',
             'birthisland', 'artisancave', 'roaminghoenn',
             'terracave', 'marinecave', 'alteringcave',
             'battleresort', 'battletower', 'farawayisland',
             'southernisland', 'teammagma/aquahideout', 'desertunderpass',
             'miragespots', 'scorchedslab', 'seamauville', 'sealedchamber',
             'fortreecity', 'route101', 'route110', 'littleroottown',
             'miragetower', 'mossdeepcity', 'lavaridgetown',
             'route119', 'rustborocity']
for location in notneeded:
    while True:
        try:
            locations.remove(location)
        except:
            break
print(locations)

['route102', 'route103', 'route104', 'route105', 'route106', 'route107', 'route108', 'route109', 'route111', 'route112', 'route113', 'route114', 'route115', 'route116', 'route117', 'route118', 'route120', 'route121', 'route122', 'route123', 'route124', 'route125', 'route126', 'route127', 'route128', 'route129', 'route130', 'route131', 'route132', 'route133', 'route134', 'abandonedship', 'caveoforigin', 'dewfordtown', 'evergrandecity', 'fierypath', 'granitecave', 'jaggedpass', 'lilycovecity', 'meteorfalls', 'mirageisland', 'mt.pyre', 'newmauville', 'pacifidlogtown', 'petalburgcity', 'petalburgwoods', 'rusturftunnel', 'safarizone', 'seafloorcavern', 'shoalcave', 'skypillar', 'slateportcity', 'sootopoliscity', 'victoryroad']


In [13]:
#convert list to csv for convenience
df = pd.DataFrame(locations)
df.to_csv('locationlist.csv', index=False)

# Pokemon List

In [ ]:
url_pkmn = 'https://pokemondb.net/pokedex/game/ruby-sapphire-emerald'
response_pkmn = requests.get(url_pkmn)
soup_pkmn = BeautifulSoup(response_pkmn.content, "html.parser")

In [44]:
#class a.ent-name in class span.infocard-lg-data text-muted in class div.infocard
tags_pkmn = soup_pkmn.find_all('a', attrs = {'class' : 'ent-name'})
pokemon = []
for pkmn in tags_pkmn:
    pokemon.append(pkmn.text.strip().lower())
print(pokemon)

['treecko', 'grovyle', 'sceptile', 'torchic', 'combusken', 'blaziken', 'mudkip', 'marshtomp', 'swampert', 'poochyena', 'mightyena', 'zigzagoon', 'linoone', 'wurmple', 'silcoon', 'beautifly', 'cascoon', 'dustox', 'lotad', 'lombre', 'ludicolo', 'seedot', 'nuzleaf', 'shiftry', 'taillow', 'swellow', 'wingull', 'pelipper', 'ralts', 'kirlia', 'gardevoir', 'surskit', 'masquerain', 'shroomish', 'breloom', 'slakoth', 'vigoroth', 'slaking', 'abra', 'kadabra', 'alakazam', 'nincada', 'ninjask', 'shedinja', 'whismur', 'loudred', 'exploud', 'makuhita', 'hariyama', 'goldeen', 'seaking', 'magikarp', 'gyarados', 'azurill', 'marill', 'azumarill', 'geodude', 'graveler', 'golem', 'nosepass', 'skitty', 'delcatty', 'zubat', 'golbat', 'crobat', 'tentacool', 'tentacruel', 'sableye', 'mawile', 'aron', 'lairon', 'aggron', 'machop', 'machoke', 'machamp', 'meditite', 'medicham', 'electrike', 'manectric', 'plusle', 'minun', 'magnemite', 'magneton', 'voltorb', 'electrode', 'volbeat', 'illumise', 'oddish', 'gloom', 

In [45]:
#delete postgame, starter or unavailable pokemon
notneededpkmn = ['treecko', 'grovyle', 'sceptile',
                 'torchic', 'combusken', 'blaziken',
                 'mudkip', 'marshtomp', 'swampert',
                 'surskit', 'masquerain', 'meditite',
                 'medicham', 'roselia', 'zangoose',
                 'lunatone', 'groudon', 'kyogre',
                 'rayquaza', 'latios', 'latias',
                 'jirachi', 'deoxys']

pokemon2 = pokemon
for pkmn in notneededpkmn:
    while True:
        try:
            pokemon2.remove(pkmn)
        except:
            break
print(pokemon2)

['poochyena', 'mightyena', 'zigzagoon', 'linoone', 'wurmple', 'silcoon', 'beautifly', 'cascoon', 'dustox', 'lotad', 'lombre', 'ludicolo', 'seedot', 'nuzleaf', 'shiftry', 'taillow', 'swellow', 'wingull', 'pelipper', 'ralts', 'kirlia', 'gardevoir', 'shroomish', 'breloom', 'slakoth', 'vigoroth', 'slaking', 'abra', 'kadabra', 'alakazam', 'nincada', 'ninjask', 'shedinja', 'whismur', 'loudred', 'exploud', 'makuhita', 'hariyama', 'goldeen', 'seaking', 'magikarp', 'gyarados', 'azurill', 'marill', 'azumarill', 'geodude', 'graveler', 'golem', 'nosepass', 'skitty', 'delcatty', 'zubat', 'golbat', 'crobat', 'tentacool', 'tentacruel', 'sableye', 'mawile', 'aron', 'lairon', 'aggron', 'machop', 'machoke', 'machamp', 'electrike', 'manectric', 'plusle', 'minun', 'magnemite', 'magneton', 'voltorb', 'electrode', 'volbeat', 'illumise', 'oddish', 'gloom', 'vileplume', 'bellossom', 'doduo', 'dodrio', 'gulpin', 'swalot', 'carvanha', 'sharpedo', 'wailmer', 'wailord', 'numel', 'camerupt', 'slugma', 'magcargo', 

In [46]:
#convert to csv
df = pd.DataFrame(pokemon2)
df.to_csv('pokemonlist.csv', index=False)

In [3]:
#further cuts to the list after making the first csv 
#pokemon only available by evolution, breeding or gift
#and irrelevant to a wild encounter table
evolved = ['beautifly', 'dustox', 'ludicolo',
           'shiftry', 'kirlia', 'gardevoir',
           'breloom', 'vigoroth', 'slaking',
           'ninjask', 'shedinja', 'exploud',
           'azurill', 'delcatty', 'aggron',
           'swalot', 'camerupt', 'grumpig',
           'vibrava', 'flygon', 'cacturne',
           'crawdaunt', 'cradily', 'armaldo',
           'milotic', 'castform', 'dusclops',
           'glalie', 'sealeo', 'walrein',
           'huntail', 'gorebyss', 'shelgon',
           'salamence', 'lileep', 'anorith',
           'beldum', 'metang', 'metagross',
           'raichu', 'sandslash', 'ninetales',
           'wigglytuff', 'vileplume', 'persian',
           'kadabra', 'alakazam', 'machoke',
           'machamp', 'golem', 'muk', 'weezing',
           'rhydon', 'seadra', 'starmie', 'crobat',
           'pichu', 'igglybuff', 'bellossom',
           'azumarill', 'magcargo', 'kingdra',
           'donphan']
pokemon2 = pokemon
for pkmn in evolved:
    while True:
        try:
            pokemon2.remove(pkmn)
        except:
            break
print(len(pokemon2))
print(pokemon2)

117
['poochyena', 'mightyena', 'zigzagoon', 'linoone', 'wurmple', 'silcoon', 'cascoon', 'lotad', 'lombre', 'seedot', 'nuzleaf', 'taillow', 'swellow', 'wingull', 'pelipper', 'ralts', 'shroomish', 'slakoth', 'abra', 'nincada', 'whismur', 'loudred', 'makuhita', 'hariyama', 'goldeen', 'seaking', 'magikarp', 'gyarados', 'marill', 'geodude', 'graveler', 'nosepass', 'skitty', 'zubat', 'golbat', 'tentacool', 'tentacruel', 'sableye', 'mawile', 'aron', 'lairon', 'machop', 'electrike', 'manectric', 'plusle', 'minun', 'magnemite', 'magneton', 'voltorb', 'electrode', 'volbeat', 'illumise', 'oddish', 'gloom', 'doduo', 'dodrio', 'gulpin', 'carvanha', 'sharpedo', 'wailmer', 'wailord', 'numel', 'slugma', 'torkoal', 'grimer', 'koffing', 'spoink', 'sandshrew', 'spinda', 'skarmory', 'trapinch', 'cacnea', 'swablu', 'altaria', 'seviper', 'solrock', 'barboach', 'whiscash', 'corphish', 'baltoy', 'claydol', 'jigglypuff', 'feebas', 'staryu', 'kecleon', 'shuppet', 'banette', 'duskull', 'tropius', 'chimecho', 'ab

In [4]:
df = pd.DataFrame(pokemon2)
df.to_csv('pokemonlist.csv', index=False)

In [12]:
#table for connecting tables that use id to tables that use name

cursor.execute('CREATE TABLE POKEMON \
                  (PKID INT NOT NULL, \
                   NAME TEXT NOT NULL);')
conn.commit()

In [13]:
#ids are 0-116 in order of list
for n in range(len(pokemon)):
     cursor.execute("INSERT INTO POKEMON (PKID, NAME) \
                     VALUES(" +str(n) +",'" +str(pokemon[n]) +"');")
conn.commit()

In [11]:
#drop table if needed for debug
cursor.execute("DROP TABLE POKEMON");
conn.commit()

In [94]:
#filter out nat dex pokemon that happened to be scraped in the encounter tables
cursor.execute("SELECT DISTINCT NAME FROM WILDS;")
wilds_pkmn = cursor.fetchall()
wilds_list = []
for tuple_list in wilds_pkmn:
    wilds_list.append(tuple_list[0])

for mon in pokemon:
    if mon in wilds_list:
        wilds_list.remove(mon)
        
print(wilds_list)

['aipom', 'teddiursa', 'ledyba', 'sunkern', 'hoothoot', 'pineco', 'houndour', 'miltank', 'shuckle', 'mareep', 'spinarak', 'gligar', 'snubbull', 'stantler', 'wooper', 'quagsire', 'remoraid', 'octillery', 'rayquaza']


In [96]:
for mon in wilds_list:
    cursor.execute("DELETE FROM WILDS WHERE NAME == '"+str(mon)+"';")
conn.commit()

# Learnset Table

In [7]:
#one learnset scrape

#make soup given pkmn name
pkmn = 'lotad'
url = 'https://pokemondb.net/pokedex/' + str(pkmn) + '/moves/3'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

#name of moves
#the soup needs to find all a tags of 'ent-name' class
#WITHIN the first instance of tbody tag
table = soup.find('tbody')
tags_moves = table.find_all('a', attrs = {'class' : 'ent-name'})
moves = []
for move in tags_moves:
    moves.append(move.text.strip().lower())
print(moves)

#we also need the cooresponding levels
#first instance of td tag of class 'cell-num', for all tr tags
tags_cells = table.find_all('tr')
tags_levels = []
for cell in tags_cells:
    level = cell.find('td', attrs = {'class' : 'cell-num'})
    tags_levels.append(level)
levels = []
for level in tags_levels:
    levels.append(int(level.text.strip()))
print(levels)

['astonish', 'growl', 'absorb', 'nature power', 'mist', 'rain dance', 'mega drain']
[1, 3, 7, 13, 21, 31, 43]


In [8]:
#must be true to continue
len(moves) == len(levels)

True

In [14]:
#conversion of moves and levels into a sqlite table

cursor.execute('CREATE TABLE pkmn_'+ str(pkmn) +' \
                  (MOVE TEXT NOT NULL, \
                   LV INT NOT NULL);')
                   
for i in range(len(moves)):
    cursor.execute("INSERT INTO pkmn_"+ str(pkmn) +" (MOVE, LV) \
                    VALUES('"+str(moves[i]) +"'," + str(levels[i]) +")")
conn.commit()


In [15]:
#verification that the table is correct

for row in cursor.execute("SELECT MOVE, LV FROM pkmn_lotad"):
    print(row[0])
    print(row[1])


astonish
1
growl
3
absorb
7
nature power
13
mist
21
rain dance
31
mega drain
43


In [16]:
#drop pkmn_lotad so it doesn't interfere with the loop
cursor.execute('DROP TABLE pkmn_lotad');
conn.commit()


In [2]:
#and now the loop of all the above

for pkmn in pokemon:

    #soup
    url = 'https://pokemondb.net/pokedex/' + str(pkmn) + '/moves/3'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    #list of moves
    table = soup.find('tbody')
    tags_moves = table.find_all('a', attrs = {'class' : 'ent-name'})
    moves = []
    for move in tags_moves:
        moves.append(move.text.strip().lower())
    
    #list of move levels
    tags_cells = table.find_all('tr')
    tags_levels = []
    for cell in tags_cells:
        level = cell.find('td', attrs = {'class' : 'cell-num'})
        tags_levels.append(level)
    levels = []
    for level in tags_levels:
        levels.append(int(level.text.strip()))
    
    #create and populate table
    cursor.execute('CREATE TABLE pkmn_'+ str(pkmn) +' \
                  (MOVE TEXT NOT NULL, \
                   LV INT NOT NULL);')               
    for i in range(len(moves)):
        cursor.execute("INSERT INTO pkmn_"+ str(pkmn) +" (MOVE, LV) \
                        VALUES('"+str(moves[i]) +"'," + str(levels[i]) +")")
    conn.commit()
    

In [18]:
#loop for resetting db if needed
for pkmn in pokemon:
    cursor.execute('DROP TABLE pkmn_'+str(pkmn));
conn.commit()

OperationalError: no such table: pkmn_lotad

In [5]:
#in which I realize all of the above was idiotic when I could be populating one table with one-to-many ids
#this new table has a connection to the db 'Emerald_Encounters'

cursor.execute('CREATE TABLE LEARNSETS \
                  (PKID INT NOT NULL, \
                   MOVE TEXT NOT NULL, \
                   LV INT NOT NULL);')
conn.commit()

In [6]:
for n in range(len(pokemon)):

    #soup
    url = 'https://pokemondb.net/pokedex/' + str(pokemon[n]) + '/moves/3'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    #list of moves
    table = soup.find('tbody')
    tags_moves = table.find_all('a', attrs = {'class' : 'ent-name'})
    moves = []
    for move in tags_moves:
        moves.append(move.text.strip().lower())
    
    #list of move levels
    tags_cells = table.find_all('tr')
    tags_levels = []
    for cell in tags_cells:
        level = cell.find('td', attrs = {'class' : 'cell-num'})
        tags_levels.append(level)
    levels = []
    for level in tags_levels:
        levels.append(int(level.text.strip()))
    
    #populate table              
    for i in range(len(moves)):
        cursor.execute("INSERT INTO LEARNSETS (PKID, MOVE, LV) \
                        VALUES("+str(n)+",'"+str(moves[i]) +"'," \
                                + str(levels[i]) +")")
    conn.commit()
    

# Encounter Table (LOCATION, NAME, LV)

In [52]:
#one location scrape

#soup
loc = 'route102'
url =  'https://www.serebii.net/pokearth/hoenn/3rd/' + str(loc) + '.shtml'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

#name of pokemon
#tag table class 'dextable' --> tag td class 'name'
#unfortunately, by finding all dextable instances, encounters are not organized by method
#the site formatting didn't allow for automated method classification

tables = soup.find_all('table', attrs = {'class' : 'dextable'})
encounters = []
for table in tables:
    tags_encounters = table.find_all('td', attrs = {'class' : 'name'})
    for name in tags_encounters:
        encounters.append(name.text.strip().lower())
print(encounters)

#max level
#tag table class 'dextable' --> tag td class 'level'
encounter_levels = []
for table in tables:
    tags_levels = table.find_all('td', attrs = {'class' : 'level'})
    for level in tags_levels:
        encounter_levels.append(int(level.text.strip()))
print(encounter_levels)

#there is no distinction between max and min level
#will have to remove even number tags (as count starts at 0)
max_levels = []
for i in range(len(encounter_levels)):
    if encounter_levels[i] == 0:
        continue
    if i % 2 != 0:
        max_levels.append(encounter_levels[i])
    else:
        continue
print(max_levels)


['wurmple', 'poochyena', 'lotad', 'zigzagoon', 'ralts', 'seedot', 'marill', 'goldeen', 'magikarp', 'goldeen', 'magikarp', 'goldeen', 'corphish', 'corphish']
[3, 4, 3, 4, 3, 4, 3, 4, 4, 4, 3, 3, 5, 35, 20, 30, 5, 10, 5, 10, 10, 30, 10, 30, 10, 30, 20, 45]
[4, 4, 4, 4, 4, 3, 35, 30, 10, 10, 30, 30, 30, 45]


In [30]:
len(max_levels) == len(encounters)

True

In [71]:
cursor.execute('CREATE TABLE WILDS \
                  (LOCATION TEXT NOT NULL, \
                   ENCOUNTER_ID INT NOT NULL, \
                   NAME TEXT NOT NULL, \
                   LV INT NOT NULL, \
                   MOVE1 TEXT, \
                   MOVE2 TEXT, \
                   MOVE3 TEXT, \
                   MOVE4 TEXT);')
conn.commit()

In [70]:
#drop wilds table if needed
cursor.execute("DROP TABLE WILDS");
conn.commit()


In [72]:
for n in range(len(locations)):
    url =  'https://www.serebii.net/pokearth/hoenn/3rd/' + str(locations[n]) + '.shtml'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    tables = soup.find_all('table', attrs = {'class' : 'dextable'})
    encounters = []
    for table in tables:
        tags_encounters = table.find_all('td', attrs = {'class' : 'name'})
        for name in tags_encounters:
            encounters.append(name.text.strip().lower())

    encounter_levels = []
    for table in tables:
        tags_levels = table.find_all('td', attrs = {'class' : 'level'})
        for level in tags_levels:
            encounter_levels.append(int(level.text.strip()))

    max_levels = []
    for i in range(len(encounter_levels)):
        if encounter_levels[i] == 0:
            continue
        if i % 2 != 0:
            max_levels.append(encounter_levels[i])
        else:
            continue
            
    #populating table
    for x in range(len(encounters)):
        cursor.execute("INSERT INTO WILDS (LOCATION, ENCOUNTER_ID, NAME, LV) \
                        VALUES('"+str(locations[n])+"'," \
                                 +str(x)+",'" \
                                 +str(encounters[x])+"'," \
                                 +str(max_levels[x])+")");
    conn.commit()


In [74]:
#manually restoring the data that couldn't be automated
          
route101 = [['route101', 'wurmple', 3],
            ['route101', 'poochyena', 3],
            ['route101', 'zigzagoon', 3]]

route110 = [['route110', 'electrike', 13],
            ['route110', 'poochyena', 12],
            ['route110', 'minun', 13],
            ['route110', 'gulpin', 13],
            ['route110', 'oddish', 13],
            ['route110', 'wingull', 12],
            ['route110', 'plusle', 13],
            ['route110', 'tentacool', 35],
            ['route110', 'wingull', 30],
            ['route110', 'pelipper', 30],
            ['route110', 'magikarp', 10],
            ['route110', 'tentacool', 10],
            ['route110', 'magikarp', 30],
            ['route110', 'tentacool', 30],
            ['route110', 'wailmer', 30],
            ['route110', 'wailmer', 45]]
            
route119 = [['route119', 'oddish', 27],
            ['route119', 'zigzagoon', 27],
            ['route119', 'linoone', 27],
            ['route119', 'tropius', 27],
            ['route119', 'kecleon', 25],
            ['route119', 'tentacool', 35],
            ['route119', 'wingull', 30],
            ['route119', 'pelipper', 30],
            ['route119', 'magikarp', 10],
            ['route119', 'goldeen', 10],
            ['route119', 'magikarp', 30],
            ['route119', 'tentacool', 30],
            ['route119', 'carvanha', 30],
            ['route119', 'feebas', 35],
            ['route119', 'carvanha', 45]]
            
miragetower = [['miragetower', 'sandshrew', 24],
               ['miragetower', 'trapinch', 24]]
               
mossdeepcity = [['mossdeepcity', 'tentacool', 35],
                ['mossdeepcity', 'wingull', 30],
                ['mossdeepcity', 'pelipper', 30],
                ['mossdeepcity', 'magikarp', 10],
                ['mossdeepcity', 'tentacool', 10],
                ['mossdeepcity', 'magikarp', 30],
                ['mossdeepcity', 'tentacool', 30],
                ['mossdeepcity', 'wailmer', 30],
                ['mossdeepcity', 'wailmer', 45],
                ['mossdeepcity', 'sharpedo', 35]]

errors = [route101, route110, route119, miragetower, mossdeepcity]

for location in errors:
    for x in range(len(location)):
        cursor.execute("INSERT INTO WILDS (LOCATION, ENCOUNTER_ID, NAME, LV) \
                        VALUES('"+str(location[x][0])+"'," \
                                 +str(x)+",'" \
                                 +str(location[x][1])+"'," \
                                 +str(location[x][2])+")");
        conn.commit()


In [75]:
#adding the used locations back in list
errors = ['route101', 'route110', 'route119', 'miragetower', 'mossdeepcity']
for loc in errors:
    locations.append(loc)
df = pd.DataFrame(locations)
df.to_csv('locationlist.csv', index=False)

# Encounter Table (MOVE1 - MOVE4)

In [76]:
len(locations)

59

In [77]:
#retrieving the moveset of one pokemon
#intended result : ['wrap', 'bubble beam', 'acid', 'constrict']
pkmn = 'tentacool'
LV = 35
LOCATION = 'mossdeepcity'
ENCOUNTER_ID = 0

cursor.execute("SELECT PKID FROM POKEMON \
                WHERE NAME =='"+str(pkmn)+"';")
PKID = cursor.fetchall()[0][0]
cursor.execute("SELECT MOVE FROM LEARNSETS \
                        WHERE PKID == "+str(PKID)+" \
                        AND LV <= "+str(LV)+" \
                        ORDER BY LV DESC LIMIT 4;")
moveset = cursor.fetchall()
#converting moveset to a list
moveset_as_list = []
for tuple_list in moveset:
    moveset_as_list.append(tuple_list[0])
    
print(moveset_as_list)

#moveset_as_list = iterated value in moveset_list
print(moveset_as_list[0])

#updating the database with moveset
cursor.execute("UPDATE WILDS \
                SET MOVE1 = '"+str(moveset_as_list[0])+"', \
                    MOVE2 = '"+str(moveset_as_list[1])+"', \
                    MOVE3 = '"+str(moveset_as_list[2])+"', \
                    MOVE4 = '"+str(moveset_as_list[3])+"' \
                WHERE LOCATION == '"+str(LOCATION)+"' \
                      AND ENCOUNTER_ID == "+str(ENCOUNTER_ID)+";")


['wrap', 'bubble beam', 'acid', 'constrict']
wrap


In [83]:
#how encounters are iterated
cursor.execute("SELECT * FROM WILDS WHERE LOCATION == 'mossdeepcity';")
encounters = cursor.fetchall()

for mon in encounters:
    LOCATION = mon[0]
    ENCOUNTER_ID = mon[1]
    NAME = mon[2]
    LV = mon[3]
    print(LOCATION, ENCOUNTER_ID, NAME, LV)

mossdeepcity 0 tentacool 35
mossdeepcity 1 wingull 30
mossdeepcity 2 pelipper 30
mossdeepcity 3 magikarp 10
mossdeepcity 4 tentacool 10
mossdeepcity 5 magikarp 30
mossdeepcity 6 tentacool 30
mossdeepcity 7 wailmer 30
mossdeepcity 8 wailmer 45
mossdeepcity 9 sharpedo 35


In [88]:
#retrieving moveset data for the pokemon of one location
#intended len of moveset_list == 10

cursor.execute("SELECT * FROM WILDS WHERE LOCATION == 'mossdeepcity';")
encounters = cursor.fetchall()

#get the moveset for each encounter
for mon in encounters:
    
    LOCATION = mon[0]
    ENCOUNTER_ID = mon[1]
    NAME = mon[2]
    LV = mon[3]
    
    cursor.execute("SELECT PKID FROM POKEMON \
                    WHERE NAME =='"+str(NAME)+"';")
    PKID = cursor.fetchall()[0][0]
    cursor.execute("SELECT MOVE FROM LEARNSETS \
                            WHERE PKID == "+str(PKID)+" \
                            AND LV <= "+str(LV)+" \
                            ORDER BY LV DESC LIMIT 4;")
    moveset = cursor.fetchall()
    
    #converting moveset to a list
    moveset_as_list = []
    for tuple_list in moveset:
        moveset_as_list.append(tuple_list[0])
        
    #add moveset to wilds
    if len(moveset_as_list) == 4:
        cursor.execute("UPDATE WILDS \
                        SET MOVE1 = '"+str(moveset_as_list[0])+"', \
                            MOVE2 = '"+str(moveset_as_list[1])+"', \
                            MOVE3 = '"+str(moveset_as_list[2])+"', \
                            MOVE4 = '"+str(moveset_as_list[3])+"' \
                        WHERE LOCATION == '"+str(LOCATION)+"' \
                              AND ENCOUNTER_ID == "+str(ENCOUNTER_ID)+";")
        conn.commit()
    elif len(moveset_as_list) == 3:
        cursor.execute("UPDATE WILDS \
                        SET MOVE1 = '"+str(moveset_as_list[0])+"', \
                            MOVE2 = '"+str(moveset_as_list[1])+"', \
                            MOVE3 = '"+str(moveset_as_list[2])+"' \
                        WHERE LOCATION == '"+str(LOCATION)+"' \
                              AND ENCOUNTER_ID == "+str(ENCOUNTER_ID)+";")
        conn.commit()
    elif len(moveset_as_list) == 2:
        cursor.execute("UPDATE WILDS \
                        SET MOVE1 = '"+str(moveset_as_list[0])+"', \
                            MOVE2 = '"+str(moveset_as_list[1])+"' \
                        WHERE LOCATION == '"+str(LOCATION)+"' \
                              AND ENCOUNTER_ID == "+str(ENCOUNTER_ID)+";")
        conn.commit()
    else:
        cursor.execute("UPDATE WILDS \
                        SET MOVE1 = '"+str(moveset_as_list[0])+"' \
                        WHERE LOCATION == '"+str(LOCATION)+"' \
                              AND ENCOUNTER_ID == "+str(ENCOUNTER_ID)+";")
        conn.commit()
    

print(moveset_list)
print(len(moveset_list))
cursor.execute("SELECT * FROM WILDS WHERE LOCATION == 'mossdeepcity';")
print(cursor.fetchall())


[['wrap', 'bubble beam', 'acid', 'constrict'], ['mist', 'wing attack', 'supersonic', 'growl'], ['protect', 'mist', 'wing attack', 'supersonic'], ['splash'], ['supersonic', 'poison sting'], ['flail', 'tackle', 'splash'], ['wrap', 'bubble beam', 'acid', 'constrict'], ['water pulse', 'astonish', 'whirlpool', 'rollout'], ['water spout', 'rest', 'mist', 'water pulse'], ['slash', 'screech', 'crunch', 'scary face']]
10
[('mossdeepcity', 0, 'tentacool', 35, 'wrap', 'bubble beam', 'acid', 'constrict'), ('mossdeepcity', 1, 'wingull', 30, 'mist', 'wing attack', 'supersonic', 'growl'), ('mossdeepcity', 2, 'pelipper', 30, 'protect', 'mist', 'wing attack', 'supersonic'), ('mossdeepcity', 3, 'magikarp', 10, 'splash', None, None, None), ('mossdeepcity', 4, 'tentacool', 10, 'supersonic', 'poison sting', None, None), ('mossdeepcity', 5, 'magikarp', 30, 'flail', 'tackle', 'splash', None), ('mossdeepcity', 6, 'tentacool', 30, 'wrap', 'bubble beam', 'acid', 'constrict'), ('mossdeepcity', 7, 'wailmer', 30, 

In [97]:
#the final loop - all locations

for loc in locations:
    
    #get encounter table for location
    cursor.execute("SELECT * FROM WILDS WHERE LOCATION == '"+str(loc)+"';")
    encounters = cursor.fetchall()

    for mon in encounters:
        
        #get moveset for a given encounter
        LOCATION = mon[0]
        ENCOUNTER_ID = mon[1]
        NAME = mon[2]
        LV = mon[3]

        cursor.execute("SELECT PKID FROM POKEMON \
                        WHERE NAME =='"+str(NAME)+"';")
        PKID = cursor.fetchall()[0][0]
        cursor.execute("SELECT MOVE FROM LEARNSETS \
                                WHERE PKID == "+str(PKID)+" \
                                AND LV <= "+str(LV)+" \
                                ORDER BY LV DESC LIMIT 4;")
        moveset = cursor.fetchall()

        #converting moveset to a list
        moveset_as_list = []
        for tuple_list in moveset:
            moveset_as_list.append(tuple_list[0])

        #add moveset to wilds
        if len(moveset_as_list) == 4:
            cursor.execute("UPDATE WILDS \
                            SET MOVE1 = '"+str(moveset_as_list[0])+"', \
                                MOVE2 = '"+str(moveset_as_list[1])+"', \
                                MOVE3 = '"+str(moveset_as_list[2])+"', \
                                MOVE4 = '"+str(moveset_as_list[3])+"' \
                            WHERE LOCATION == '"+str(LOCATION)+"' \
                                  AND ENCOUNTER_ID == "+str(ENCOUNTER_ID)+";")
            conn.commit()
        elif len(moveset_as_list) == 3:
            cursor.execute("UPDATE WILDS \
                            SET MOVE1 = '"+str(moveset_as_list[0])+"', \
                                MOVE2 = '"+str(moveset_as_list[1])+"', \
                                MOVE3 = '"+str(moveset_as_list[2])+"' \
                            WHERE LOCATION == '"+str(LOCATION)+"' \
                                  AND ENCOUNTER_ID == "+str(ENCOUNTER_ID)+";")
            conn.commit()
        elif len(moveset_as_list) == 2:
            cursor.execute("UPDATE WILDS \
                            SET MOVE1 = '"+str(moveset_as_list[0])+"', \
                                MOVE2 = '"+str(moveset_as_list[1])+"' \
                            WHERE LOCATION == '"+str(LOCATION)+"' \
                                  AND ENCOUNTER_ID == "+str(ENCOUNTER_ID)+";")
            conn.commit()
        else:
            cursor.execute("UPDATE WILDS \
                            SET MOVE1 = '"+str(moveset_as_list[0])+"' \
                            WHERE LOCATION == '"+str(LOCATION)+"' \
                                  AND ENCOUNTER_ID == "+str(ENCOUNTER_ID)+";")
            conn.commit()
